In [9]:
import ipynb.fs.full.helper as h

from imp import reload
reload(h)

<module 'ipynb.fs.full.helper' (/tf/src/scripts/helper.ipynb)>

In [2]:
import os, cv2, shutil, random
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pandas as pd
import numpy as np
from deepface import DeepFace
from retinaface import RetinaFace

In [3]:
# get unknown pictures e.g. from one of those datasets
# https://analyticsindiamag.com/10-face-datasets-to-start-facial-recognition-projects/
# and unpack them in the /images/unknown/original folder

In [12]:
original = h.unknownOriginal
cropped = h.unknownCropped

myFiles = [f for f in os.listdir(original) if f.endswith('.jpg')]
    
for file in range(len(myFiles)):
    
    file_name = myFiles[file]
    try:         
        img = cv2.imread(original + "/" + file_name)
        faces = RetinaFace.detect_faces(img)
            
        for i in range(len(faces)):
            face = faces[f'face_{i+1}']['facial_area']
            (xmin,ymin,xmax,ymax) = face
            crop = img[ymin:ymax,xmin:xmax]
            crop = cv2.resize(crop,(224,224))
                
        plt.imsave(cropped + "/" + file_name, cv2.cvtColor(crop, cv2.COLOR_RGB2BGR))
        
    except ValueError:
        print(ValueError)
        print(file_name)
            
    except TypeError:
        print(file_name)

In [13]:
    imageFolder = h.unknownCropped
    folder = 'Unknown'

    myFiles = [f for f in os.listdir(imageFolder) if f.endswith('.jpg')]
    random.shuffle(myFiles)
    
    for i, file in enumerate(myFiles):
        if(i <= int(len(myFiles) * 0.8)):
            if not os.path.exists(f'{h.celebTrain}/{folder}'):
                os.mkdir(f'{h.celebTrain}/{folder}')
            shutil.copy(f'{imageFolder}/{file}', f'{h.celebTrain}/{folder}/{file}')
        else:
            if not os.path.exists(f'{h.celebTest}/{folder}'):
                os.mkdir(f'{h.celebTest}/{folder}')
            shutil.copy(f'{imageFolder}/{file}', f'{h.celebTest}/{folder}/{file}')

In [ ]:
##############################################

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
rows=h.img_size  #Number of Pixels in the Rows for Input. 
cols=h.img_size  #Number of Pixels in Columns for Output

In [ ]:
trained_model=MobileNetV2(input_shape=(rows,cols,3),           
                    include_top=False,
                    weights='imagenet')   

#Un-Freeze all the pretrained layers of 'MobileNetV2 for Training.
trained_model.trainable=True 
trained_model.summary()

In [ ]:
last_layer=trained_model.get_layer('out_relu')   
last_layer_output=last_layer.output 

In [ ]:
# augmentation in train dataset
train_datagen = ImageDataGenerator(rescale=1/255,
                                shear_range=0.2,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                rotation_range=20,
                                width_shift_range=0.1,
                                height_shift_range=0.1)
                                 
test_datagen = ImageDataGenerator(rescale=1/255)

train_generator=train_datagen.flow_from_directory(h.celebTrain,
                                                  target_size=(rows,cols),
                                                  class_mode='categorical')

test_generator=test_datagen.flow_from_directory(h.celebTest,
                                                target_size=(rows,cols),
                                                class_mode='categorical')

In [ ]:
epochs = 10
dropout = 0.2

In [ ]:
x = tf.keras.layers.GlobalAveragePooling2D()(last_layer_output)

#Add a Dropout layer.
x = tf.keras.layers.Dropout(dropout)(x)

# Add a final softmax layer for classification.
x = tf.keras.layers.Dense(h.celeb_nbr,activation='softmax')(x)

model = tf.keras.Model(trained_model.input,x) 

model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])
#Summary of the model.
model.summary()

In [ ]:
# model training
result=model.fit(train_generator,
                 validation_data=test_generator,
                 epochs=epochs,
                 verbose=1)

In [ ]:
acc = result.history['accuracy']
val_acc = result.history['val_accuracy']

loss = result.history['loss']
val_loss = result.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(9, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
model.save(h.modelsVCeleb)

In [ ]:
model.evaluate(test_generator)